In [ ]:
import keras
from keras.models import load_model, Model, Input
from keras.layers import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras import backend as K
from keras.utils import plot_model

from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from statsmodels.robust import mad
import random
import cv2
from IPython.display import clear_output

Using TensorFlow backend.


In [ ]:
imsize = (192, 192, 3) # Image dimensions
numdim = 30 # Number of latent dimensions

def sample(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal((K.shape(z_mu)[0], numdim),)
    return z_mu + K.exp(z_log_sigma) * epsilon

class CustomLossLayer(keras.layers.Layer):
    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        
        # Reconstruction loss
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        
        # KL divergence
        kl_loss = -5e-4 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
        
        return K.mean(xent_loss + kl_loss)        

    # Adds the custom loss
    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        return z_decoded


In [ ]:
input_img = Input(imsize)

x = Conv2D(32, 4,
          activation = 'relu',
          strides = 2,
          padding = 'same')(input_img)

x = Conv2D(64, 4,
          activation = 'relu',
          strides = 2,
          padding = 'same')(x)

x = Conv2D(128, 4,
          activation = 'relu',
          strides = 2,
          padding = 'same')(x)

x = Conv2D(128, 4,
          activation='relu',
          padding = 'same')(x)

x = Conv2D(128, 4,
          activation = 'relu',
          padding = 'same')(x)

x = Conv2D(256, 4,
          activation = 'relu',
          padding='same')(x)

x = Conv2D(256, 4,
          activation = 'relu',
          padding = 'same')(x)

x = Conv2D(256, (1,4),
          activation = 'relu',
          padding = 'same')(x)

b4shape = K.int_shape(x)

x = Flatten()(x)
z_mu = Dense(numdim)(x)
z_log_sigma = Dense(numdim)(x)
z = Lambda(sample)([z_mu, z_log_sigma]) # Combining z_mu and z_log_sigma
z = BatchNormalization()(z)

encoder = Model(input_img, z) # Assigns half the VAE

decoder_input = Input(K.int_shape(z)[1:])

x = Dense(np.prod(b4shape[1:]),
          activation = 'relu')(decoder_input)

x = Reshape(b4shape[1:])(x)

x = Conv2DTranspose(256, (4,1), 
                  activation = 'relu',
                  padding='same')(x)

x = Conv2DTranspose(256, 4, 
                  activation = 'relu',
                  padding='same')(x)

x = Conv2DTranspose(256, 4, 
                  activation = 'relu',
                  padding='same')(x)

x = Conv2DTranspose(256, 4, 
                  activation = 'relu',
                  padding = 'same')(x)

x = Conv2DTranspose(128, 4, 
                  activation = 'relu',
                  padding = 'same',)(x)

x = Conv2DTranspose(128, 4,  
                  activation = 'relu',
                  padding = 'same')(x)

x = Conv2DTranspose(64, 4,  
                  activation = 'relu',
                  padding = 'same')(x)

x = Conv2DTranspose(32, 4,  
                 activation = 'relu',
                 padding = 'same')(x)

x = Conv2DTranspose(24, 4,  
                  activation = 'relu',
                  padding = 'same',
                   strides = 2)(x)

x = Conv2DTranspose(9, 4,  
                  activation = 'relu',
                  padding = 'same',
                  strides = 2)(x)

x = Conv2DTranspose(3, 6, 
          padding = 'same', 
          activation = 'sigmoid',
          strides = 2)(x)

decoder = Model(decoder_input, x) # Assigns half the VAE

z_decoded = decoder(z)

y = CustomLossLayer()([input_img, z_decoded]) # Adds loss

def no_loss(y_true, y_pred): # Because loss is given in the custom layer
    return y_pred-y_pred

vae = Model(input_img, y) # Assigns the combined encoder-decoder model
vae.compile(optimizer=Adam(lr=0.00146), loss=no_loss) 

plot_model(vae, 'vae_architecture.png', show_shapes = True) # Plots model architecture to a file
plot_model(decoder, 'decoder_architecture.png', show_shapes = True)

print('AUTOENCODER ARCHITECTURE ("model" is the decoder)')
vae.summary()
print('DECODER ARCHITECTURE')
decoder.summary()


In [ ]:
vae.load_weights('vae_192_30.hdf5')

In [ ]:
# vae.save('vae_192_30.h5py')
# vae.save_weights('vae_w_192_30.hdf5')

In [ ]:
# image feeder for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,)
train_generator = train_datagen.flow_from_directory(
        'manyfaces', # Folder name with all the images. Inside the folder should be ANOTHER folder with images
        target_size=imsize[:-1],
        batch_size=16,)


In [ ]:
#training
epochs = 0
while True:
    vae.fit_generator(
            train_generator,
            steps_per_epoch=16054//16,
            epochs=1,)
    epochs +=1
    print(epochs)
    vae.save('vae_192_30.h5py')
    vae.save_weights('vae_w_192_30.hdf5')

In [ ]:
# Youtube channel Code Parade explains this in his video on generating faces.
def save_PCA_stuff(rand_vecs, iters):
    x_enc = encoder.predict_generator(
            train_generator,
            steps=13233//16,)

    x_mean = np.mean(x_enc, axis=0)
    x_stds = np.std(x_enc, axis=0)
    x_cov = np.cov((x_enc - x_mean).T)
    e, v = np.linalg.eig(x_cov)

    np.save('means.npy', x_mean)
    np.save('stds.npy', x_stds)
    np.save('evals.npy', e)
    np.save('evecs.npy', v)

In [ ]:
#save_PCA_stuff()

In [ ]:
'''
This cell creates a 16x9 grid of nearly 300 faces and saves it to a PNG file while also showing it
'''

# Loading PCA stuff
x_mean = np.load('means.npy',)
x_stds = np.load('stds.npy', )
e = np.load('evals.npy', )
v = np.load('evecs.npy', )

fig = plt.figure(figsize=(16,9), dpi = 194)
plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
for i in range(1,16*9+1):
        ax = plt.subplot(9, 16, i)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        vector = np.array([np.random.uniform(-2, 2) for i in range(30)])
        vector = x_mean + np.dot(v, (vector * e).T).T
        vector*=6.1
        np.clip(vector, -0.45, 0.45, vector)
        plt.imshow(decoder.predict(vector.reshape(1,30)).reshape(192,192,3))
plt.savefig('ManyFaces1.PNG', bbox_inches=0)

In [ ]:
'''
This cell generates a random face. Press any keyboard button to make a new face. Quickly press one button and then press Q to exit
'''
# Loading PCA stuff
x_mean = np.load('means.npy',)
x_stds = np.load('stds.npy', )
e = np.load('evals.npy', )
v = np.load('evecs.npy', )
while True:
    vector = np.array([np.random.uniform(-2, 2) for i in range(30)])
    vector = x_mean + np.dot(v, (vector * e).T).T
    vector*=6.1
    np.clip(vector, -0.5, 0.5, out=vector)
    
    print(vector.max(), vector.min(),  vector.mean(), mad(vector, axis=0))
    clear_output(True)
    
    image = (decoder.predict(vector.reshape(1,numdim)).reshape(*imsize)*255).astype('uint8')
    
    cv2.imshow('image', cv2.cvtColor(cv2.resize(image, (600,600)), cv2.COLOR_RGB2BGR))
    plt.show()
    if cv2.waitKey(100)&0xFF == ord('q'):
        break
    if cv2.waitKey(0):
        continue
    
cv2.destroyAllWindows()

In [ ]:
'''
This cell is for a face editor with sliders and stuff so you can make custom faces. I recommend downloading a program that keeps the face window on top so u can see the changes taking place. Press Q to exit
'''

# Loading PCA stuff
x_mean = np.load('means.npy', )
x_stds = np.load('stds.npy', )
e = np.load('evals.npy', )
v = np.load('evecs.npy', )

# The next few lines make trackbars in descending order of effect on the image
enumer = list(enumerate(e.tolist()))
enumer = pd.DataFrame(enumer)
effect_indexes = np.array(enumer.sort_values(1, ascending=False))[:,0].astype('int').tolist()
nums = [str(i) for i in range(numdim)]

def no(x): # Uselsess function needed for cv2.createTaskbar
    pass
cv2.namedWindow('sliders')

for i in effect_indexes:
    cv2.createTrackbar(str(i),'sliders', random.randint(350, 650), 1000, no)

while True:
    feedin = []
    for i in nums:
        feedin.append(norm.ppf((cv2.getTrackbarPos(i,'sliders')+1)/1002))
    feedin = np.array(feedin)
    feedin *= 15
    feedin = x_mean + np.dot(v, (feedin * e).T).T
    
    #np.clip(feedin, -0.5, 0.5, feedin)
    feedin = feedin.reshape(1,numdim)
    image = decoder.predict(feedin)
    image = (image.reshape(*imsize)*255).astype('uint8')
    
    print(feedin.max(), feedin.min(), feedin.mean(), *mad(feedin, axis=1), sep='\n')
    print(feedin.round(1))
    clear_output(True)
    
    cv2.imshow('image', cv2.cvtColor(cv2.resize(image, (500,500)), cv2.COLOR_RGB2BGR))
    if cv2.waitKey(1)&0xFF == ord('q'):
        break
cv2.destroyAllWindows()

In [ ]:
'''
This cell creates video of a smoothly changing random face. press Q to exit
'''


x_mean = np.load('means.npy',)
x_stds = np.load('stds.npy', )
e = np.load('evals.npy', )
v = np.load('evecs.npy', )


# Initializes a random vector
feedin = np.zeros((1,numdim))
for i in range(numdim):
    feedin[:,i] = np.random.uniform(-0.1,0.1)
add = np.zeros((1,numdim))
for i in range(numdim):
    add[:,i]+=np.random.uniform(0.1, -0.1)

while True:
    # slightly change the vector
    for i in range(numdim):
        add[:,i]+=np.random.uniform(-0.6*add.max(),-0.6*add.min()) 
        if np.random.uniform(-0.5, 1)<0: # 33 % chance each value in the vector becomes 5% smaller
            add[:, i]*=0.95
            
    np.clip(add, -0.5, 0.5, add)
    for i in range(numdim):
        feedin[:, i]+=add[:,i]
        if np.random.uniform(-0.5, 1)<0: # 33 % chance each value in the vector becomes 5% smaller
            feedin[:, i]*=0.95
        if feedin[:, i] in [feedin.max(), feedin.min()]: # decreases the smallest and largest value by 5%
            feedin[:, i] *= 0.95 
            
    # do some fancy PCA stuff on the vector to make faces look better. 'n' stands for 'new'
    nfeedin = x_mean + np.dot(v, (feedin * e).T).T
    nfeedin*=1
    np.clip(nfeedin, -0.2, 0.2, nfeedin)
    clear_output(True)
    feedin -= feedin.mean()

    print(nfeedin.max(), nfeedin.min(), nfeedin.mean(), *mad(nfeedin, axis=1), sep='\n') #info on whats being fed into the decoder
    print(nfeedin.round(2))
    
    image = decoder.predict(nfeedin)
    image = (image.reshape(*imsize)*255).astype('uint8')
    image = cv2.cvtColor(cv2.resize(image, (192*2,192*2), interpolation = cv2.INTER_LANCZOS4), cv2.COLOR_RGB2BGR)
    cv2.imshow('image', image)
    if cv2.waitKey(15)&0xFF == ord('q'): # Decrease value to make changing faster
        break
cv2.destroyAllWindows()